<a href="https://colab.research.google.com/github/sonu2012/Information-Retrieval/blob/master/Information_Retrieval_SONU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import numpy as np
import pandas as pd
import re
import nltk
import spacy
import string
pd.options.mode.chained_assignment = None
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/content/drive/MyDrive/NLP/archive'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        with open(os.path.join(dirname, filename)) as f:
            line_count = 0
            id_set = set()
            for l in f.readlines():
                line_count += 1
                if filename == "CISI.REL":
                    id_set.add(l.lstrip(" ").split(" ")[0])
                elif l.startswith(".I "):
                    id_set.add(l.split(" ")[1].strip())
            print(f"{filename} : {len(id_set)} items, over {line_count} lines.")

/content/drive/MyDrive/NLP/archive/CISI.ALL
CISI.ALL : 1460 items, over 108747 lines.
/content/drive/MyDrive/NLP/archive/CISI.QRY
CISI.QRY : 112 items, over 1525 lines.
/content/drive/MyDrive/NLP/archive/CISI.REL
CISI.REL : 76 items, over 3114 lines.


In [2]:
with open('/content/drive/MyDrive/NLP/archive/CISI.ALL') as f:
    lines = ""
    for l in f.readlines():
        lines += "\n" + l.strip() if l.startswith(".") else " " + l.strip()
    lines = lines.lstrip("\n").split("\n")
    
# print n lines
n = 10
for l in lines[:n]:
    print(l)

.I 1
.T 18 Editions of the Dewey Decimal Classifications
.A Comaromi, J.P.
.W The present study is a history of the DEWEY Decimal Classification.  The first edition of the DDC was published in 1876, the eighteenth edition in 1971, and future editions will continue to appear as needed.  In spite of the DDC's long and healthy life, however, its full story has never been told.  There have been biographies of Dewey that briefly describe his system, but this is the first attempt to provide a detailed history of the work that more than any other has spurred the growth of librarianship in this country and abroad.
.X 1	5	1 92	1	1 262	1	1 556	1	1 1004	1	1 1024	1	1 1024	1	1
.I 2
.T Use Made of Technical Libraries
.A Slater, M.
.W This report is an analysis of 6300 acts of use in 104 technical libraries in the United Kingdom. Library use is only one aspect of the wider pattern of information use.  Information transfer in libraries is restricted to the use of documents.  It takes no account of doc

In [3]:
doc_set = {}
doc_id = ""
doc_text = ""
for l in lines:
    if l.startswith(".I"):
        doc_id = l.split(" ")[1].strip()
    elif l.startswith(".X"):
        doc_set[doc_id] = doc_text.lstrip(" ")
        doc_id = ""
        doc_text = ""
    else:
        doc_text += l.strip()[3:] + " " # The first 3 characters of a line can be ignored.

# Print something to see the dictionary structure, etc.
print(f"Number of documents = {len(doc_set)}" + ".\n")
print(doc_set["5"]) # note that the dictionary indexes are strings, not numbers. 
        

Number of documents = 1460.

A Library Management Game: a report on a research project Brophy, P. Although the use of games in professional education has become widespread only during the last decade, the method has been used in a number of fields for many hundreds of years. Its origins have been traced to simple war games, used in military training when the "real thing" was either unavailable or too dangerous.  In more recent times, these games have become more and more sophisticated, and many now use large electronic computers to handle the complex calculations involved. Since 1956, when the first well-developed management game was introduced, the technique has spread rapidly into a wide variety of disciplines and today it is used at all levels of education, from primary school classes to courses for experienced professional men and women.  One of the main causes of this "game explosion" has been the rapid development of sophisticated management techniques, such as simulation and mat

In [4]:
with open('/content/drive/MyDrive/NLP/archive/CISI.QRY') as f:
    lines = ""
    for l in f.readlines():
        lines += "\n" + l.strip() if l.startswith(".") else " " + l.strip()
    lines = lines.lstrip("\n").split("\n")
    
qry_set = {}
qry_id = ""
for l in lines:
    if l.startswith(".I"):
        qry_id = l.split(" ")[1].strip()
    elif l.startswith(".W"):
        qry_set[qry_id] = l.strip()[3:]
        qry_id = ""
    
# Print something to see the dictionary structure, etc.
print(f"Number of queries = {len(qry_set)}" + ".\n")
print(qry_set["3"]) # note that the dictionary indexes are strings, not numbers.

Number of queries = 112.

What is information science?  Give definitions where possible.


In [5]:
rel_set = {}
with open('/content/drive/MyDrive/NLP/archive/CISI.REL') as f:
    for l in f.readlines():
        qry_id = l.lstrip(" ").strip("\n").split("\t")[0].split(" ")[0]
        doc_id = l.lstrip(" ").strip("\n").split("\t")[0].split(" ")[-1]
        if qry_id in rel_set:
            rel_set[qry_id].append(doc_id)
        else:
            rel_set[qry_id] = []
            rel_set[qry_id].append(doc_id)
        if qry_id == "7":
            print(l.strip("\n"))
    
# Print something to see the dictionary structure, etc.
print(f"\nNumber of mappings = {len(rel_set)}" + ".\n")
print(rel_set["7"]) # note that the dictionary indexes are strings, not numbers

     7    310	0	0.000000
     7    320	0	0.000000
     7    332	0	0.000000
     7    375	0	0.000000
     7    376	0	0.000000
     7    645	0	0.000000
     7    724	0	0.000000
     7    725	0	0.000000

Number of mappings = 76.

['310', '320', '332', '375', '376', '645', '724', '725']


In [6]:
docs = []
for i in range(1,len(doc_set)+1):
    docs.append(doc_set[str(i)])
    dfdoc=pd.DataFrame(docs,columns={'Docs'})

In [7]:
print(dfdoc)

                                                   Docs
0     18 Editions of the Dewey Decimal Classificatio...
1     Use Made of Technical Libraries Slater, M. Thi...
2     Two Kinds of Power An Essay on Bibliographic C...
3     Systems Analysis of a University Library; fina...
4     A Library Management Game: a report on a resea...
...                                                 ...
1455  World Dynamics Forrester, J.W. Over the last s...
1456  World Trends in Library Education Bramley, G. ...
1457  Legal Restrictions on Exploitation of the Pate...
1458  Language and Thought Poluskin, V.A. This book ...
1459  Modern Integral Information Systems for Chemis...

[1460 rows x 1 columns]


In [8]:
dfdoc["text_lower"] = dfdoc["Docs"].str.lower()
dfdoc.head()

,Docs,text_lower
0,18 Editions of the Dewey Decimal Classificatio...,18 editions of the dewey decimal classificatio...
1,"Use Made of Technical Libraries Slater, M. Thi...","use made of technical libraries slater, m. thi..."
2,Two Kinds of Power An Essay on Bibliographic C...,two kinds of power an essay on bibliographic c...
3,Systems Analysis of a University Library; fina...,systems analysis of a university library; fina...
4,A Library Management Game: a report on a resea...,a library management game: a report on a resea...


In [9]:
# drop the new column created in last cell
dfdoc.drop(["text_lower"], axis=1, inplace=True)

PUNCT_TO_REMOVE = string.punctuation
def remove_punctuation(text):
    """custom function to remove the punctuation"""
    return text.translate(str.maketrans('', '', PUNCT_TO_REMOVE))

dfdoc["text_wo_punct"] = dfdoc["Docs"].apply(lambda text: remove_punctuation(text))
dfdoc.head()

,Docs,text_wo_punct
0,18 Editions of the Dewey Decimal Classificatio...,18 Editions of the Dewey Decimal Classificatio...
1,"Use Made of Technical Libraries Slater, M. Thi...",Use Made of Technical Libraries Slater M This ...
2,Two Kinds of Power An Essay on Bibliographic C...,Two Kinds of Power An Essay on Bibliographic C...
3,Systems Analysis of a University Library; fina...,Systems Analysis of a University Library final...
4,A Library Management Game: a report on a resea...,A Library Management Game a report on a resear...


In [10]:
nltk.download('stopwords')
from nltk.corpus import stopwords
", ".join(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


"i, me, my, myself, we, our, ours, ourselves, you, you're, you've, you'll, you'd, your, yours, yourself, yourselves, he, him, his, himself, she, she's, her, hers, herself, it, it's, its, itself, they, them, their, theirs, themselves, what, which, who, whom, this, that, that'll, these, those, am, is, are, was, were, be, been, being, have, has, had, having, do, does, did, doing, a, an, the, and, but, if, or, because, as, until, while, of, at, by, for, with, about, against, between, into, through, during, before, after, above, below, to, from, up, down, in, out, on, off, over, under, again, further, then, once, here, there, when, where, why, how, all, any, both, each, few, more, most, other, some, such, no, nor, not, only, own, same, so, than, too, very, s, t, can, will, just, don, don't, should, should've, now, d, ll, m, o, re, ve, y, ain, aren, aren't, couldn, couldn't, didn, didn't, doesn, doesn't, hadn, hadn't, hasn, hasn't, haven, haven't, isn, isn't, ma, mightn, mightn't, mustn, mus

In [11]:
STOPWORDS = set(stopwords.words('english'))
def remove_stopwords(text):
    """custom function to remove the stopwords"""
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])

dfdoc["text_wo_stop"] = dfdoc["text_wo_punct"].apply(lambda text: remove_stopwords(text))
dfdoc.head()

,Docs,text_wo_punct,text_wo_stop
0,18 Editions of the Dewey Decimal Classificatio...,18 Editions of the Dewey Decimal Classificatio...,18 Editions Dewey Decimal Classifications Coma...
1,"Use Made of Technical Libraries Slater, M. Thi...",Use Made of Technical Libraries Slater M This ...,Use Made Technical Libraries Slater M This rep...
2,Two Kinds of Power An Essay on Bibliographic C...,Two Kinds of Power An Essay on Bibliographic C...,Two Kinds Power An Essay Bibliographic Control...
3,Systems Analysis of a University Library; fina...,Systems Analysis of a University Library final...,Systems Analysis University Library final repo...
4,A Library Management Game: a report on a resea...,A Library Management Game a report on a resear...,A Library Management Game report research proj...


In [12]:
dfdoc["text_to_lower"] = dfdoc["text_wo_stop"].str.lower()
dfdoc.head(10)

,Docs,text_wo_punct,text_wo_stop,text_to_lower
0,18 Editions of the Dewey Decimal Classificatio...,18 Editions of the Dewey Decimal Classificatio...,18 Editions Dewey Decimal Classifications Coma...,18 editions dewey decimal classifications coma...
1,"Use Made of Technical Libraries Slater, M. Thi...",Use Made of Technical Libraries Slater M This ...,Use Made Technical Libraries Slater M This rep...,use made technical libraries slater m this rep...
2,Two Kinds of Power An Essay on Bibliographic C...,Two Kinds of Power An Essay on Bibliographic C...,Two Kinds Power An Essay Bibliographic Control...,two kinds power an essay bibliographic control...
3,Systems Analysis of a University Library; fina...,Systems Analysis of a University Library final...,Systems Analysis University Library final repo...,systems analysis university library final repo...
4,A Library Management Game: a report on a resea...,A Library Management Game a report on a resear...,A Library Management Game report research proj...,a library management game report research proj...
5,"Abstracting Concepts and Methods Borko, H. Gra...",Abstracting Concepts and Methods Borko H Gradu...,Abstracting Concepts Methods Borko H Graduate ...,abstracting concepts methods borko h graduate ...
6,Academic Library Buildings A Guide to Architec...,Academic Library Buildings A Guide to Architec...,Academic Library Buildings A Guide Architectur...,academic library buildings a guide architectur...
7,The Academic Library Essays in Honor of Guy R....,The Academic Library Essays in Honor of Guy R ...,The Academic Library Essays Honor Guy R Lyle F...,the academic library essays honor guy r lyle f...
8,"Access to Libraries in College Hyman, R.T. Thi...",Access to Libraries in College Hyman RT This s...,Access Libraries College Hyman RT This study a...,access libraries college hyman rt this study a...
9,"Access to Periodical Resources Palmour, V.E. T...",Access to Periodical Resources Palmour VE The ...,Access Periodical Resources Palmour VE The pur...,access periodical resources palmour ve the pur...


In [13]:
from collections import Counter
cnt = Counter()
for text in dfdoc["text_to_lower"].values:
    for word in text.split():
        cnt[word] += 1
        
cnt.most_common(10)

[('the', 2090),
 ('information', 1573),
 ('library', 1207),
 ('system', 751),
 ('a', 727),
 ('libraries', 611),
 ('data', 586),
 ('research', 565),
 ('retrieval', 550),
 ('this', 534)]

In [14]:
FREQWORDS = set([w for (w, wc) in cnt.most_common(10)])
def remove_freqwords(text):
    """custom function to remove the frequent words"""
    return " ".join([word for word in str(text).split() if word not in FREQWORDS])

dfdoc["text_wo_stopfreq"] = dfdoc["text_to_lower"].apply(lambda text: remove_freqwords(text))
dfdoc.head()

,Docs,text_wo_punct,text_wo_stop,text_to_lower,text_wo_stopfreq
0,18 Editions of the Dewey Decimal Classificatio...,18 Editions of the Dewey Decimal Classificatio...,18 Editions Dewey Decimal Classifications Coma...,18 editions dewey decimal classifications coma...,18 editions dewey decimal classifications coma...
1,"Use Made of Technical Libraries Slater, M. Thi...",Use Made of Technical Libraries Slater M This ...,Use Made Technical Libraries Slater M This rep...,use made technical libraries slater m this rep...,use made technical slater m report analysis 63...
2,Two Kinds of Power An Essay on Bibliographic C...,Two Kinds of Power An Essay on Bibliographic C...,Two Kinds Power An Essay Bibliographic Control...,two kinds power an essay bibliographic control...,two kinds power an essay bibliographic control...
3,Systems Analysis of a University Library; fina...,Systems Analysis of a University Library final...,Systems Analysis University Library final repo...,systems analysis university library final repo...,systems analysis university final report proje...
4,A Library Management Game: a report on a resea...,A Library Management Game a report on a resear...,A Library Management Game report research proj...,a library management game report research proj...,management game report project brophy p althou...


In [15]:
# Drop the two columns which are no more needed 
dfdoc.drop(["text_wo_punct", "text_to_lower"], axis=1, inplace=True)

n_rare_words = 10
RAREWORDS = set([w for (w, wc) in cnt.most_common()[:-n_rare_words-1:-1]])
def remove_rarewords(text):
    """custom function to remove the rare words"""
    return " ".join([word for word in str(text).split() if word not in RAREWORDS])

dfdoc["text_wo_stopfreqrare"] = dfdoc["text_wo_stopfreq"].apply(lambda text: remove_rarewords(text))
dfdoc.head()

,Docs,text_wo_stop,text_wo_stopfreq,text_wo_stopfreqrare
0,18 Editions of the Dewey Decimal Classificatio...,18 Editions Dewey Decimal Classifications Coma...,18 editions dewey decimal classifications coma...,18 editions dewey decimal classifications coma...
1,"Use Made of Technical Libraries Slater, M. Thi...",Use Made Technical Libraries Slater M This rep...,use made technical slater m report analysis 63...,use made technical slater m report analysis 63...
2,Two Kinds of Power An Essay on Bibliographic C...,Two Kinds Power An Essay Bibliographic Control...,two kinds power an essay bibliographic control...,two kinds power an essay bibliographic control...
3,Systems Analysis of a University Library; fina...,Systems Analysis University Library final repo...,systems analysis university final report proje...,systems analysis university final report proje...
4,A Library Management Game: a report on a resea...,A Library Management Game report research proj...,management game report project brophy p althou...,management game report project brophy p althou...


In [16]:
from nltk.stem.porter import PorterStemmer

# Drop the two columns 
dfdoc.drop(["text_wo_stopfreq"], axis=1, inplace=True) 

stemmer = PorterStemmer()
def stem_words(text):
    return " ".join([stemmer.stem(word) for word in text.split()])

dfdoc["text_stemmed"] = dfdoc["text_wo_stopfreqrare"].apply(lambda text: stem_words(text))
dfdoc.head()

,Docs,text_wo_stop,text_wo_stopfreqrare,text_stemmed
0,18 Editions of the Dewey Decimal Classificatio...,18 Editions Dewey Decimal Classifications Coma...,18 editions dewey decimal classifications coma...,18 edit dewey decim classif comaromi jp presen...
1,"Use Made of Technical Libraries Slater, M. Thi...",Use Made Technical Libraries Slater M This rep...,use made technical slater m report analysis 63...,use made technic slater m report analysi 6300 ...
2,Two Kinds of Power An Essay on Bibliographic C...,Two Kinds Power An Essay Bibliographic Control...,two kinds power an essay bibliographic control...,two kind power an essay bibliograph control wi...
3,Systems Analysis of a University Library; fina...,Systems Analysis University Library final repo...,systems analysis university final report proje...,system analysi univers final report project bu...
4,A Library Management Game: a report on a resea...,A Library Management Game report research proj...,management game report project brophy p althou...,manag game report project brophi p although us...


In [17]:
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()
def lemmatize_words(text):
    return " ".join([lemmatizer.lemmatize(word) for word in text.split()])

dfdoc["text_lemmatized"] = dfdoc["text_wo_stopfreqrare"].apply(lambda text: lemmatize_words(text))
dfdoc.head()

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


,Docs,text_wo_stop,text_wo_stopfreqrare,text_stemmed,text_lemmatized
0,18 Editions of the Dewey Decimal Classificatio...,18 Editions Dewey Decimal Classifications Coma...,18 editions dewey decimal classifications coma...,18 edit dewey decim classif comaromi jp presen...,18 edition dewey decimal classification comaro...
1,"Use Made of Technical Libraries Slater, M. Thi...",Use Made Technical Libraries Slater M This rep...,use made technical slater m report analysis 63...,use made technic slater m report analysi 6300 ...,use made technical slater m report analysis 63...
2,Two Kinds of Power An Essay on Bibliographic C...,Two Kinds Power An Essay Bibliographic Control...,two kinds power an essay bibliographic control...,two kind power an essay bibliograph control wi...,two kind power an essay bibliographic control ...
3,Systems Analysis of a University Library; fina...,Systems Analysis University Library final repo...,systems analysis university final report proje...,system analysi univers final report project bu...,system analysis university final report projec...
4,A Library Management Game: a report on a resea...,A Library Management Game report research proj...,management game report project brophy p althou...,manag game report project brophi p although us...,management game report project brophy p althou...


In [18]:
datasethoc_mesh=pd.DataFrame(dfdoc["text_lemmatized"])

In [19]:
datasethoc_mesh

,text_lemmatized
0,18 edition dewey decimal classification comaro...
1,use made technical slater m report analysis 63...
2,two kind power an essay bibliographic control ...
3,system analysis university final report projec...
4,management game report project brophy p althou...
...,...
1455,world dynamic forrester jw over last several d...
1456,world trend education bramley g one significan...
1457,legal restriction exploitation patent monopoly...
1458,language thought va book considers basic aspec...


In [20]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

In [21]:
def splitbagorwords(text):
  document =text
  bagofwords=document.split(' ')
  return bagofwords

In [22]:
datasethoc_mesh['bagG']=datasethoc_mesh['text_lemmatized'].apply(lambda row: splitbagorwords(row))

In [23]:
datasethoc_mesh

,text_lemmatized,bagG
0,18 edition dewey decimal classification comaro...,"[18, edition, dewey, decimal, classification, ..."
1,use made technical slater m report analysis 63...,"[use, made, technical, slater, m, report, anal..."
2,two kind power an essay bibliographic control ...,"[two, kind, power, an, essay, bibliographic, c..."
3,system analysis university final report projec...,"[system, analysis, university, final, report, ..."
4,management game report project brophy p althou...,"[management, game, report, project, brophy, p,..."
...,...,...
1455,world dynamic forrester jw over last several d...,"[world, dynamic, forrester, jw, over, last, se..."
1456,world trend education bramley g one significan...,"[world, trend, education, bramley, g, one, sig..."
1457,legal restriction exploitation patent monopoly...,"[legal, restriction, exploitation, patent, mon..."
1458,language thought va book considers basic aspec...,"[language, thought, va, book, considers, basic..."


In [24]:
def uniq(text1):
  return set(text1)

In [25]:
datasethoc_mesh['uniquewordscc']=datasethoc_mesh['bagG'].apply(lambda row: uniq(row))

In [26]:
datasethoc_mesh

,text_lemmatized,bagG,uniquewordscc
0,18 edition dewey decimal classification comaro...,"[18, edition, dewey, decimal, classification, ...","{briefly, 1876, spurred, needed, future, long,..."
1,use made technical slater m report analysis 63...,"[use, made, technical, slater, m, report, anal...","{technical, m, visit, act, receive, current, m..."
2,two kind power an essay bibliographic control ...,"[two, kind, power, an, essay, bibliographic, c...","{writing, great, storehouse, story, stock, con..."
3,system analysis university final report projec...,"[system, analysis, university, final, report, ...","{buckland, stressed, ugc, need, questioned, an..."
4,management game report project brophy p althou...,"[management, game, report, project, brophy, p,...","{unavailable, origin, today, war, since, advan..."
...,...,...,...
1455,world dynamic forrester jw over last several d...,"[world, dynamic, forrester, jw, over, last, se...","{separate, show, toward, but, force, populatio..."
1456,world trend education bramley g one significan...,"[world, trend, education, bramley, g, one, sig...","{trace, evolution, upon, comparing, future, de..."
1457,legal restriction exploitation patent monopoly...,"[legal, restriction, exploitation, patent, mon...","{thus, securingto, wa, remarkable, art, invent..."
1458,language thought va book considers basic aspec...,"[language, thought, va, book, considers, basic...","{evolution, language, generalization, complex,..."


In [27]:
def numwords(uniqtext,bagw):
  numOfWords = dict.fromkeys(uniqtext, 0)
  for word in bagw:
    numOfWords[word] += 1
  return numOfWords

In [28]:
datasethoc_mesh["numwords"]=datasethoc_mesh.apply(lambda row:numwords(row['uniquewordscc'],row['bagG']),axis=1)

In [29]:
datasethoc_mesh

,text_lemmatized,bagG,uniquewordscc,numwords
0,18 edition dewey decimal classification comaro...,"[18, edition, dewey, decimal, classification, ...","{briefly, 1876, spurred, needed, future, long,...","{'briefly': 1, '1876': 1, 'spurred': 1, 'neede..."
1,use made technical slater m report analysis 63...,"[use, made, technical, slater, m, report, anal...","{technical, m, visit, act, receive, current, m...","{'technical': 4, 'm': 1, 'visit': 1, 'act': 2,..."
2,two kind power an essay bibliographic control ...,"[two, kind, power, an, essay, bibliographic, c...","{writing, great, storehouse, story, stock, con...","{'writing': 4, 'great': 1, 'storehouse': 1, 's..."
3,system analysis university final report projec...,"[system, analysis, university, final, report, ...","{buckland, stressed, ugc, need, questioned, an...","{'buckland': 1, 'stressed': 1, 'ugc': 1, 'need..."
4,management game report project brophy p althou...,"[management, game, report, project, brophy, p,...","{unavailable, origin, today, war, since, advan...","{'unavailable': 1, 'origin': 1, 'today': 1, 'w..."
...,...,...,...,...
1455,world dynamic forrester jw over last several d...,"[world, dynamic, forrester, jw, over, last, se...","{separate, show, toward, but, force, populatio...","{'separate': 2, 'show': 1, 'toward': 1, 'but':..."
1456,world trend education bramley g one significan...,"[world, trend, education, bramley, g, one, sig...","{trace, evolution, upon, comparing, future, de...","{'trace': 1, 'evolution': 1, 'upon': 1, 'compa..."
1457,legal restriction exploitation patent monopoly...,"[legal, restriction, exploitation, patent, mon...","{thus, securingto, wa, remarkable, art, invent...","{'thus': 1, 'securingto': 1, 'wa': 1, 'remarka..."
1458,language thought va book considers basic aspec...,"[language, thought, va, book, considers, basic...","{evolution, language, generalization, complex,...","{'evolution': 1, 'language': 2, 'generalizatio..."


In [30]:
def computeTF(wordDict, bagOfWords):
    tfDict = {}
    bagOfWordsCount = len(bagOfWords)
    for word, count in wordDict.items():
        tfDict[word] = count / float(bagOfWordsCount)
    return tfDict

In [31]:
datasethoc_mesh["tf"]=datasethoc_mesh.apply(lambda row:computeTF(row['numwords'],row['bagG']),axis=1)

In [37]:
datasethoc_mesh.head(10)

,text_lemmatized,bagG,uniquewordscc,numwords,tf
0,18 edition dewey decimal classification comaro...,"[18, edition, dewey, decimal, classification, ...","{briefly, 1876, spurred, needed, future, long,...","{'briefly': 1, '1876': 1, 'spurred': 1, 'neede...","{'briefly': 0.018867924528301886, '1876': 0.01..."
1,use made technical slater m report analysis 63...,"[use, made, technical, slater, m, report, anal...","{technical, m, visit, act, receive, current, m...","{'technical': 4, 'm': 1, 'visit': 1, 'act': 2,...","{'technical': 0.05333333333333334, 'm': 0.0133..."
2,two kind power an essay bibliographic control ...,"[two, kind, power, an, essay, bibliographic, c...","{writing, great, storehouse, story, stock, con...","{'writing': 4, 'great': 1, 'storehouse': 1, 's...","{'writing': 0.058823529411764705, 'great': 0.0..."
3,system analysis university final report projec...,"[system, analysis, university, final, report, ...","{buckland, stressed, ugc, need, questioned, an...","{'buckland': 1, 'stressed': 1, 'ugc': 1, 'need...","{'buckland': 0.018867924528301886, 'stressed':..."
4,management game report project brophy p althou...,"[management, game, report, project, brophy, p,...","{unavailable, origin, today, war, since, advan...","{'unavailable': 1, 'origin': 1, 'today': 1, 'w...","{'unavailable': 0.010526315789473684, 'origin'..."
5,abstracting concept method borko h graduate sc...,"[abstracting, concept, method, borko, h, gradu...","{topic, containing, need, graduate, text, part...","{'topic': 2, 'containing': 1, 'need': 1, 'grad...","{'topic': 0.014705882352941176, 'containing': ..."
6,academic building guide architectural issue so...,"[academic, building, guide, architectural, iss...","{show, mistake, solution, academic, done, eval...","{'show': 1, 'mistake': 1, 'solution': 3, 'acad...","{'show': 0.019230769230769232, 'mistake': 0.01..."
7,academic essay honor guy r lyle farber ei a im...,"[academic, essay, honor, guy, r, lyle, farber,...","{administration, thrustthe, professor, impress...","{'administration': 1, 'thrustthe': 1, 'profess...","{'administration': 0.009523809523809525, 'thru..."
8,access college hyman rt study assumed addition...,"[access, college, hyman, rt, study, assumed, a...","{hyman, need, 1890, guidance, assumed, analyti...","{'hyman': 1, 'need': 1, '1890': 1, 'guidance':...","{'hyman': 0.01098901098901099, 'need': 0.01098..."
9,access periodical resource palmour ve purpose ...,"[access, periodical, resource, palmour, ve, pu...","{loan, evaluate, confined, request, storing, n...","{'loan': 4, 'evaluate': 1, 'confined': 1, 'req...","{'loan': 0.0449438202247191, 'evaluate': 0.011..."


In [33]:
datasethoc_mesh.to_csv("featurematrixlemm.csv")

In [34]:
from google.colab import files
files.download('featurematrixlemm.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>